# Autoprocessing on stoomboot

j.angevaare@nikhef.nl

03-12-2021

## Goal
In this notebook, we will start many jobs to process all the available data of `XAMSL`

In [1]:
import amstrax
import numpy as np

2021-12-03 17:02:22,704 - utilix - WARNING - Could not load a configuration file. You can create one at /user/jorana/.xenon_config, or set a custom path using

export XENON_CONFIG=path/to/your/config



In case you haven't set your cridentials yet in your `.bashrc` ([follow instructions](https://amstrax.readthedocs.io/en/latest/setup.html#setting-up-amstrax)) you might have to run something like this :

```python
import os

os.environ.update(dict(
    DAQ_PASSWORD='???',
    MONGO_USER='???',
    MONGO_PASSWORD='???',
))
```


Let's have a look at the available runs for this context:

In [4]:
st = amstrax.contexts.xams_little()

[amstrax.rundb.RunDB, readonly: True, strax.storage.files.DataDirectory, readonly: True, path: /data/xenon/xamsl/raw/, take_only: ('raw_records_v1724', 'raw_records_v1730', 'raw_records_aqmon'), strax.storage.files.DataDirectory, readonly: True, path: /data/xenon/xamsl/processed/, strax.storage.files.DataDirectory, path: ./amstrax_data]


In [5]:
runs = st.select_runs()
runs

Fetching run info from MongoDB: 100%|████████████████████████████████████████████| 2462/2462 [00:00<00:00, 16409.28it/s]


Checking data availability:   0%|          | 0/2 [00:00<?, ?it/s]

Option n_tpc_pmts not taken by any registered plugin


,name,number,mode,start,end,tags,livetime,processing_status,raw_records_v1730_available,raw_records_v1724_available
0,000000,0,V1724_Run0,2020-11-23 08:36:02.740,2020-11-23 08:36:50.846,,0 days 00:00:48.106000,NaN,False,False
1,000001,1,V1724_Run0,2020-11-24 07:51:24.798,2020-11-24 07:51:36.827,,0 days 00:00:12.029000,NaN,False,False
2,000002,2,V1724_Run0,2020-11-24 12:55:19.577,2020-11-24 12:55:28.594,,0 days 00:00:09.017000,NaN,False,False
3,000003,3,V1724_Run0,2020-11-24 13:08:10.073,2020-11-24 13:08:19.093,,0 days 00:00:09.020000,NaN,False,False
4,000004,4,V1730_Run3,2020-11-24 13:54:39.023,2020-11-24 13:55:00.072,,0 days 00:00:21.049000,NaN,False,False
...,...,...,...,...,...,...,...,...,...,...
2457,002457,2457,test_run_two_digitizers,2021-12-03 14:22:42.380,2021-12-03 14:23:20.762,,0 days 00:00:38.382000,NaN,False,False
2458,002458,2458,test_run_two_digitizers,2021-12-03 14:23:42.734,2021-12-03 14:24:30.291,,0 days 00:00:47.557000,NaN,False,False
2459,002459,2459,test_run_two_digitizers,2021-12-03 14:24:46.061,2021-12-03 14:25:33.615,,0 days 00:00:47.554000,NaN,False,False
2460,002460,2460,test_run_two_digitizers,2021-12-03 14:25:49.426,2021-12-03 14:28:35.106,,0 days 00:02:45.680000,NaN,False,False


In [6]:
st.storage

[amstrax.rundb.RunDB, readonly: True,
 strax.storage.files.DataDirectory, readonly: True, path: /data/xenon/xamsl/raw/, take_only: ('raw_records_v1724', 'raw_records_v1730', 'raw_records_aqmon'),
 strax.storage.files.DataDirectory, readonly: True, path: /data/xenon/xamsl/processed/,
 strax.storage.files.DataDirectory, path: ./amstrax_data]

## Setting runs ready for processing:
Below, we are going to set the status of runs to 'pending' if:
 - they aren't stored yet
 - live data exists

In [7]:
rundb = amstrax.get_mongo_collection()

In [8]:
not_stored_runs = runs['name'][~runs['raw_records_v1730_available']]
stored_live_data = os.listdir(st.config['live_data_dir'])
runs_to_update = not_stored_runs[np.in1d(not_stored_runs, stored_live_data)]
print(f' doing these runs:\n{list(runs_to_update)}')

82      000082
83      000083
84      000084
85      000085
86      000086
         ...  
2436    002436
2437    002437
2438    002438
2439    002439
2440    002440
Name: name, Length: 1754, dtype: object

### Change this if you actually want to set all the processing statusses to `True`

In [11]:
change_rundocs = False

In [12]:
for r in runs_to_update:
    print(r, flush=True, end = '\r')
    if change_rundocs:
        rundb.find_one_and_update({'number': int(r)},
                                  {"$set": {"processing_status": "pending"}})

# Submit jobs

Below we submit jobs until everything with `{"processing_status": "pending"}` is done

In [ ]:
!python /data/xenon/xamsl/software/amstrax/amstrax/auto_processing/auto_processing.py

Starting autoprocess version 2.1.0...
2021-12-03 17:06:32,026 - utilix - WARNING - Could not load a configuration file. You can create one at /user/jorana/.xenon_config, or set a custom path using

export XENON_CONFIG=path/to/your/config

I found 1757 runs to process, time to get to work!
These runs I will do:
[82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 116, 117, 118, 119, 155, 163, 166, 173, 178, 180, 187, 197, 198, 199, 200, 201, 202, 203, 206, 207, 219, 266, 272, 352, 360, 369, 370, 371, 381, 390, 410, 428, 491, 492, 516, 541, 601, 619, 628, 629, 630, 631, 632, 633, 634, 638, 639, 643, 644, 650, 672, 673, 679, 680, 681, 687, 688, 689, 690, 691, 692, 693, 694, 695, 696, 697, 698, 699, 700, 701, 702, 736, 768, 769, 770, 771, 772, 773, 774, 775, 776, 777, 778, 779, 780, 781, 782, 783, 784, 785, 786, 787, 788, 789, 790, 791, 792, 794, 795, 796, 797, 798, 799, 800, 801, 802, 803, 804, 805, 806, 807, 808, 809, 810, 811, 812, 813, 814, 815, 816, 817, 818, 819, 820, 821, 822, 823, 

9308627.burrell.nikhef.nl
Submitted job for run 000082:raw_records_aqmon
qsub /data/xenon/xamsl/data_management/sync/./jobs/p_000083_raw_records_aqmon.sh -e /data/xenon/xamsl/data_management/sync/./logs/p_000083_raw_records_aqmon.log -o /data/xenon/xamsl/data_management/sync/./logs/p_000083_raw_records_aqmon.log
9308628.burrell.nikhef.nl
Submitted job for run 000083:raw_records_aqmon
qsub /data/xenon/xamsl/data_management/sync/./jobs/p_000084_raw_records_aqmon.sh -e /data/xenon/xamsl/data_management/sync/./logs/p_000084_raw_records_aqmon.log -o /data/xenon/xamsl/data_management/sync/./logs/p_000084_raw_records_aqmon.log
9308629.burrell.nikhef.nl
Submitted job for run 000084:raw_records_aqmon
qsub /data/xenon/xamsl/data_management/sync/./jobs/p_000085_raw_records_aqmon.sh -e /data/xenon/xamsl/data_management/sync/./logs/p_000085_raw_records_aqmon.log -o /data/xenon/xamsl/data_management/sync/./logs/p_000085_raw_records_aqmon.log
9308630.burrell.nikhef.nl
Submitted job for run 000085:raw

qsub /data/xenon/xamsl/data_management/sync/./jobs/p_000200_raw_records_aqmon.sh -e /data/xenon/xamsl/data_management/sync/./logs/p_000200_raw_records_aqmon.log -o /data/xenon/xamsl/data_management/sync/./logs/p_000200_raw_records_aqmon.log
9308654.burrell.nikhef.nl
Submitted job for run 000200:raw_records_aqmon
qsub /data/xenon/xamsl/data_management/sync/./jobs/p_000201_raw_records_aqmon.sh -e /data/xenon/xamsl/data_management/sync/./logs/p_000201_raw_records_aqmon.log -o /data/xenon/xamsl/data_management/sync/./logs/p_000201_raw_records_aqmon.log
9308655.burrell.nikhef.nl
Submitted job for run 000201:raw_records_aqmon
qsub /data/xenon/xamsl/data_management/sync/./jobs/p_000202_raw_records_aqmon.sh -e /data/xenon/xamsl/data_management/sync/./logs/p_000202_raw_records_aqmon.log -o /data/xenon/xamsl/data_management/sync/./logs/p_000202_raw_records_aqmon.log
9308656.burrell.nikhef.nl
Submitted job for run 000202:raw_records_aqmon
qsub /data/xenon/xamsl/data_management/sync/./jobs/p_00020

9308680.burrell.nikhef.nl
Submitted job for run 000630:raw_records_aqmon
qsub /data/xenon/xamsl/data_management/sync/./jobs/p_000631_raw_records_aqmon.sh -e /data/xenon/xamsl/data_management/sync/./logs/p_000631_raw_records_aqmon.log -o /data/xenon/xamsl/data_management/sync/./logs/p_000631_raw_records_aqmon.log
9308681.burrell.nikhef.nl
Submitted job for run 000631:raw_records_aqmon
qsub /data/xenon/xamsl/data_management/sync/./jobs/p_000632_raw_records_aqmon.sh -e /data/xenon/xamsl/data_management/sync/./logs/p_000632_raw_records_aqmon.log -o /data/xenon/xamsl/data_management/sync/./logs/p_000632_raw_records_aqmon.log
9308682.burrell.nikhef.nl
Submitted job for run 000632:raw_records_aqmon
qsub /data/xenon/xamsl/data_management/sync/./jobs/p_000633_raw_records_aqmon.sh -e /data/xenon/xamsl/data_management/sync/./logs/p_000633_raw_records_aqmon.log -o /data/xenon/xamsl/data_management/sync/./logs/p_000633_raw_records_aqmon.log
9308683.burrell.nikhef.nl
Submitted job for run 000633:raw

qsub /data/xenon/xamsl/data_management/sync/./jobs/p_000699_raw_records_aqmon.sh -e /data/xenon/xamsl/data_management/sync/./logs/p_000699_raw_records_aqmon.log -o /data/xenon/xamsl/data_management/sync/./logs/p_000699_raw_records_aqmon.log
9308707.burrell.nikhef.nl
Submitted job for run 000699:raw_records_aqmon
qsub /data/xenon/xamsl/data_management/sync/./jobs/p_000700_raw_records_aqmon.sh -e /data/xenon/xamsl/data_management/sync/./logs/p_000700_raw_records_aqmon.log -o /data/xenon/xamsl/data_management/sync/./logs/p_000700_raw_records_aqmon.log
9308708.burrell.nikhef.nl
Submitted job for run 000700:raw_records_aqmon
qsub /data/xenon/xamsl/data_management/sync/./jobs/p_000701_raw_records_aqmon.sh -e /data/xenon/xamsl/data_management/sync/./logs/p_000701_raw_records_aqmon.log -o /data/xenon/xamsl/data_management/sync/./logs/p_000701_raw_records_aqmon.log
9308709.burrell.nikhef.nl
Submitted job for run 000701:raw_records_aqmon
qsub /data/xenon/xamsl/data_management/sync/./jobs/p_00070

In [41]:
def j_id(j):
    if j.split():
        if j.split()[0].split('.'):
            ji=j.split()[0].split('.')[0]
            if len(ji)==7 and ji != 'burrell':
                return ji
    return False

In [42]:
qs = !qstat -u jorana


['9258064',
 '9258065',
 '9258066',
 '9258067',
 '9258068',
 '9258069',
 '9258070',
 '9258071',
 '9288859',
 '9288861',
 '9288862',
 '9288864',
 '9288868',
 '9288869',
 '9288871',
 '9288872',
 '9288873',
 '9288875',
 '9288876',
 '9288880',
 '9288881',
 '9288882',
 '9288883',
 '9288885',
 '9288886',
 '9288887',
 '9288888',
 '9288889',
 '9288890',
 '9288891',
 '9288893',
 '9288894',
 '9288895',
 '9288910',
 '9288915',
 '9288920',
 '9304075',
 '9307491',
 '9307492',
 '9307493',
 '9307494',
 '9307495',
 '9307496',
 '9307497',
 '9307498',
 '9307499',
 '9307500',
 '9307501',
 '9307502',
 '9307503',
 '9307504',
 '9307506',
 '9307507',
 '9307508',
 '9307509',
 '9307510',
 '9307511',
 '9307512',
 '9307513',
 '9307514',
 '9307515',
 '9307516',
 '9307517',
 '9307518',
 '9307519',
 '9307520',
 '9307521',
 '9307522',
 '9307523',
 '9307524',
 '9307525',
 '9307526',
 '9307527',
 '9307528',
 '9307529',
 '9307530',
 '9307531',
 '9307532',
 '9307533',
 '9307534',
 '9307535',
 '9307536',
 '9307537',
 '93

In [43]:
# for ji in [j_id(j) for j in qs if j_id(j)]:
#     !qdel $ji